In [ ]:
#전체 이미지폴더 가져와서 하나의 폴더로 만들기
#만든 하나의 폴더에서 xml만 추출해서 xml파일만 저장하는 폴더 만들고 저장하기
#xml파일들 모두 불러서 bollard만 추출하고 이미지들에 대한 txt파일 만들기
#빈 txt파일과 그 파일에 해당하는 이미지 파일 삭제하기

In [22]:
#전체 이미지폴더 가져와서 하나의 폴더로 만들기 - 30여초 소요 :)
import shutil # shutil 모듈은 파일과 파일 모음에 대한 여러 가지 고수준 연산을 제공
import time
import os
import glob
import pandas as pd
    
# 저장할 폴더 생성하기
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error:Creating directory.'+directory)
createFolder('/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/images')

#여러 폴더 내의 이미지들을 읽어오고 복사하는 함수 만들기
def read_all_file(path):
    output = os.listdir(path)
    file_list = []

    for i in output:
        if os.path.isdir(path+"/"+i):
            file_list.extend(read_all_file(path+"/"+i))
        elif os.path.isfile(path+"/"+i):
            file_list.append(path+"/"+i)#append는 x 그 자체를 원소로 넣고 extend는 iterable의 각 항목들을 넣음

    return file_list

def copy_all_file(file_list, new_path):
    for src_path in file_list:
        file = src_path.split("/")[-1]
        shutil.copyfile(src_path, new_path+"/"+file)
        # print("파일 {} 작업 완료".format(file)) # 작업한 파일명 출력
        
#start_time = time.time() # 작업 시작 시간 

# 이미지 가져올 폴더와 저장할 폴더 지정
src_path='/Users/ejju/Desktop/개발자대회_WISET/인도보행 영상_0811/바운딩박스/Bbox_1_new' # 기존 폴더 경로
new_path = '/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/images' # 옮길 폴더 경로

file_list = read_all_file(src_path)
copy_all_file(file_list, new_path)

# print("=" * 40)
# print("러닝 타임 : {}".format(time.time() - start_time)) # 총 소요시간 계산

In [43]:
#위에서 모두 한 폴더에 넣은 AIhub 데이터 모두 불러오기(image파일 & xml파일)
file_list=os.listdir(new_path)

#xml파일만 이동할 폴더 생성하기
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print('Error:Creating directory.'+directory)
createFolder('/Users/ejju/Desktop/개발자대회_WISET/ju/Bigdata_AIhub/xml_files')

des_path='/Users/ejju/Desktop/개발자대회_WISET/ju/Bigdata_AIhub/xml_files'

#전체 데이터에서 xml파일만 추출하기
xml_files=[]
for file in file_list:
    if(file.endswith('.xml')):
        xml_files.append(file)
        
for xml_file in xml_files:
    shutil.move(new_path+'/'+xml_file, des_path+'/'+xml_file )

In [44]:
#xml파일들 모두 불러서 bollard만 추출하고 이미지들에 대한 txt파일 만들기(bollard가 없는 이미지에 대해서는 빈 txt파일이 생성될 것임)
#30여초 소요됨
import pandas as pd
import glob

# xml파일 모두 불러오기
file_list=glob.glob('/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/xml_files/*.xml')
file_list=sorted(file_list)

# 모든 xml파일에서 데이터 가져오기
for file in file_list:
    import xml.etree.ElementTree as ET
    tree = ET.parse(file)
    root = tree.getroot()
    
    
    rows=[]

    for i in range(2,len(root)):
        id=root[i].attrib['id']
        name=root[i].attrib['name']
        x=root[i] #중요!!! 이전 파일에서 x를 선언해주지 않아서 x.attrib관련 수치들이 랜덤으로 들어왔었음
        for j in range(len(x)):
            label = x[j].attrib['label']
            xtl = x[j].attrib['xtl']
            ytl = x[j].attrib['ytl']
            xbr = x[j].attrib['xbr']
            ybr = x[j].attrib['ybr']
            rows.append({
                "id": id,
                "name": name,
                "label": label,
                "xtl": xtl,
                "ytl": ytl,
                "xbr": xbr,
                "ybr": ybr,
                })
    
    # 데이터프레임으로 보기
    df = pd.DataFrame(rows)
    
    # bollard가 있는 행만 남기기
    condition = df['label'] == 'bollard'
    df=df[condition]
    
    # 문자열로 표현되어 있는 좌표 표시 수치를 숫자형으로 변환-팀장님은 타입변환없이 진행한 거 보면 roboflow에서는 문자열로 표현된 숫자도 인식하는듯????
    #df[['xtl','ytl','xbr','ybr']]=df[['xtl','ytl','xbr','ybr']].apply(pd.to_numeric)

    # 여러 txt파일이 생성되므로 먼저 저장할 폴더 생성하기
    def createFolder(directory):
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
        except OSError:
            print('Error:Creating directory.'+directory)
    createFolder('/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/bollard_class')

    # 위 셀에서 한 이미지파일에 대해 하나의 txt파일 만든 것을 반복문에 이용하여 모든 이미지파일에 적용하여 각각의 txt파일 만들기
    # 파일 이름들이 저장되는 리스트 만들기
    file_names=[]
    for i in range(2, len(root)):
        name=root[i].attrib['name']
        file_names.append(name)

    # 각 이미지파일에 해당하는 행들끼리 나눠주기
    df_txt_files=[]
    for i in range(len(file_names)):
        df_image=(df['name']==file_names[i])
        df_txt_file=df[df_image]
        df_txt_files.append(df_txt_file)

    # 각 txt파일에 들어갈 컬럼들만 추출    
    df_columns=[]
    for i in range(len(df_txt_files)):
        df_column=df_txt_files[i][['label','xtl','ytl','xbr','ybr']]
        df_columns.append(df_column)

    # 각 이미지이름과 동일하도록 txt파일 이름 정하고, 각 txt파일에 들어갈 컬럼들만 추출한 것으로 txt파일 만들기
    for i in range(len(file_names)):
        file_name=file_names[i].split('.')[0]
        df_columns[i].to_csv('/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/bollard_class/'+file_name+'.txt', sep=' ', index=False, header=None)

In [46]:
#30여 초 소요!!!!!!!!!!!
#빈 txt파일과 그 파일에 해당하는 이미지 파일 삭제하기

# txt파일이 들어있는 폴더로 image파일도 이동
def read_all_file(path):
    output = os.listdir(path)
    file_list = []

    for i in output:
        if os.path.isdir(path+"/"+i):
            file_list.extend(read_all_file(path+"/"+i))
        elif os.path.isfile(path+"/"+i):
            file_list.append(path+"/"+i)#append는 x 그 자체를 원소로 넣고 extend는 iterable의 각 항목들을 넣음

    return file_list

def copy_all_file(file_list, new_path):
    for src_path in file_list:
        file = src_path.split("/")[-1]
        shutil.copyfile(src_path, new_path+"/"+file)
        # print("파일 {} 작업 완료".format(file)) # 작업한 파일명 출력
        
#start_time = time.time() # 작업 시작 시간 

# 이미지 가져올 폴더와 저장할 폴더 지정
src_path='/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/images' # 기존 폴더 경로
new_path = '/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/bollard_class' # 옮길 폴더 경로

file_list = read_all_file(src_path)
copy_all_file(file_list, new_path)

In [47]:
#폴더 내 bollard image가 몇 개있는지 확인하기
txt_files=glob.glob('/Users/ejju/Desktop/개발자대회_WISET/ju/bigData_AIhub/bollard_class/*.txt')
txt_files=sorted(txt_files)
print('Total number of images:',len(txt_files))

count=0
for file in txt_files:
    if os.path.getsize(file)==0:       
        count+=1
        
print('Number of empty file:',count)
print('Number of bollard image:',len(txt_files)-count)


# 빈 txt파일 찾고 빈 파일은 삭제
# 삭제된 파일과 이름이 같은 .jpg파일도 삭제하기 
for file in txt_files:
    if os.path.getsize(file)==0:       
        os.remove(file)
        name=file.split('.')[0]
        os.remove(name+'.jpg')
        
        
# 최종적으로 폴더 내에는 bollard를 포함한 이미지와 그 이미지에 대한 txt파일만 남음

Total number of images: 21522
Number of empty file: 15821
Number of bollard image: 5701
